In [1]:
include("state_utils.jl")

load_patients (generic function with 1 method)

In [2]:
state_vectors_file = apath * "state_vectors.json" 
C = restore_state_vectors(state_vectors_file)

49×750 Array{Float64,2}:
  0.40622    -0.359482    0.355052    0.421069  …   0.299349     -0.0744802
 -0.403457    0.227875    0.135838   -0.337671     -0.295816      0.0895853
  0.057937   -0.0427263  -0.0962289   0.424601      0.254819      0.189519
  0.323062    0.553472   -0.39423     0.918227      0.206701      0.421109
  0.262369    0.770682   -0.353434    0.967542      0.14065       0.475746
  0.387567    0.27039    -0.223944    1.01172   …   0.341578      0.267425
  0.0540531   0.38316    -0.452211    0.706943      0.010038      0.476519
 -0.0318675   0.837272   -0.236756   -0.310497      0.180282      0.445651
  0.0200429  -1.47475     0.4939      0.500315     -0.122612      0.233625
  0.066215   -1.72938    -0.169337    0.17992      -0.000744307  -0.115629
  0.166773   -1.86715    -0.466514   -0.228327  …   0.13         -0.171712
  0.0883901  -1.03962     0.480752    0.390763      0.0648888     0.127738
 -0.0419525   2.38272     1.47009     0.687139      0.0411181     0.15320

In [3]:
patients = load_patients();

In [4]:
patients[1]

Dict{String,Any} with 8 entries:
  "num_observations" => 74
  "mortality"        => true
  "patient_index"    => 1
  "actions"          => Any[21, 21, 1, 1, 1, 1, 1, 1, 1, 21  …  1, 1, 1, 1, 2, …
  "icuid"            => 200003
  "MAP"              => Any[83, 84.5, 68.2, 69.3333, 70, 73, 66, 68, 71, 72  … …
  "cont_actions"     => Any[Any[0.2245, 0.224, 0, 0, 0, 0, 0, 0, 0, 0.225  …  0…
  "trajectory"       => Any[Any[-0.359482, 0.439907, -1.05909, -0.487919, -0.17…

In [5]:
for pid = 1:n_patients
    p       = patients[pid]
    states  = []
    rewards = []
    tr      = p["trajectory"]
    tr_len  = size(tr)[1] 
    dims    = size(tr[1])[1]

    for j  = 1:tr_len
        d    = Float64(Inf)
        idx  = -1
        tr_k = convert(Array{Float64, 1}, tr[j])
        for k = 1:n_states
            d_k = distance(tr_k, C[:, k], dims)
            if d_k < d
                d   = d_k
                idx = k
            end
        end
        push!(states, idx)
        push!(rewards, 0.0)
    end

    if p["mortality"]
        push!(states, STATE_DECEASED)
        push!(rewards, -1.0)
    else
        push!(states, STATE_SURVIVED)
        push!(rewards, 1.0)
    end    

    mdp_p = Dict()
    mdp_p["trajectory"]      = states
    mdp_p["rewards"]         = rewards
    mdp_p["actions"]         = p["actions"]
    mdp_p["cont_actions"]    = p["cont_actions"]
    mdp_p["cont_trajectory"] = p["trajectory"]

    json_mdp_p = JSON.json(mdp_p)
    filename   = apath * "mdp_patient_" * string(pid) * ".json"

    open(filename, "w") do io
       println(io, json_mdp_p)
    end
end
